In [1]:
# | default_exp classes.DomoDataflow

In [2]:
# | exporti
from fastcore.basics import patch_to

from enum import Enum
from dataclasses import dataclass, field

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.dataflow as dataflow_routes

In [3]:
# | exporti


class DomoDataflow_Action_Type(Enum):
    LoadFromVault = "LoadFromVault"
    PublishToVault = "PublishToVault"
    GenerateTableAction = "GenerateTableAction"


@dataclass
class DomoDataflow_Action:
    type: str
    id: str
    name: str
    data_source_id: str
    sql: str

    @classmethod
    def _from_obj(cls, obj: dict):
        dd = obj
        if isinstance(dd, dict):
            dd = util_dd.DictDot(obj)

        tbl_name = dd.dataSource.name if dd.dataSource else None
        ds_id = dd.dataSource.guid if dd.dataSource else None

        return cls(
            type=dd.type,
            id=dd.id,
            name=dd.name or dd.targetTableName or dd.tableName or tbl_name,
            data_source_id=dd.dataSourceId or ds_id,
            sql=dd.selectStatement or dd.query,
        )

#### sample implementation of DataFlow_Action

Note: this list of properties that can be attached to `DomoDataflow_Action` is by no means comprehensive. To capture more data from the API, simply add properties to the class and extend the `DomoDataflow_Action._from_obj` private method.


In [4]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await dataflow_routes.get_dataflow_by_id(dataflow_id=70, auth=token_auth)

[
    DomoDataflow_Action._from_obj(action).__dict__
    for action in res.response.get("actions")
]

[{'type': 'LoadFromVault',
  'id': '241025d7-3cca-4369-b7c0-b3264277c0e1',
  'name': 'domostats_people',
  'data_source_id': '241025d7-3cca-4369-b7c0-b3264277c0e1',
  'sql': None},
 {'type': 'GenerateTableAction',
  'id': 'c176aa53-122a-4a2e-8b6f-41ca4a72d67c',
  'name': 't_1',
  'data_source_id': None,
  'sql': 'SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end numChildren\n\n\nfrom `domostats_people` p\n\n, (select @row:= 0,@rowCounter:= 1, @iter := 1, @isReset := 0 , @group := 0) r'},
 {'type': 'GenerateTableAction',
  'id': 'ac9f630d-d865-4925-b955-4316e227229a',
  'name': 'reportsto',
  'data_source_id': None,
  'sql': "\nselect  b.`User ID` as userID , 'reportsTo' as Action , a.`User ID` as bodyID\n\

In [5]:
# | export
@dataclass
class DomoDataflow:
    id: str
    name: str = None
    auth: dmda.DomoAuth = field(default=None)
    owner: str = None
    description: str = None
    tags: list[str] = None
    actions: list[DomoDataflow_Action] = None

In [6]:
# | exporti
@patch_to(DomoDataflow, cls_method=True)
async def get_from_id(
    cls: DomoDataflow,
    dataflow_id: int,
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    return_raw: bool = False,
):
    res = await dataflow_routes.get_dataflow_by_id(
        auth=auth, dataflow_id=dataflow_id, debug_api=debug_api
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    dd = util_dd.DictDot(res.response)
    domo_dataflow = cls(
        auth=auth,
        id=dd.id,
        name=dd.name,
        description=dd.description,
        owner=dd.owner,
        tags=dd.tags,
    )

    if dd.actions:
        domo_dataflow.actions = [
            DomoDataflow_Action._from_obj(action) for action in dd.actions
        ]

    return domo_dataflow

#### sample implementation of get_from_id


In [7]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDataflow.get_from_id(dataflow_id=70, auth=token_auth)

DomoDataflow(id=70, name='DomoStats - people - reports', auth=DomoTokenAuth(domo_instance='domo-community', token_name='token_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'), owner=None, description=None, tags=None, actions=[DomoDataflow_Action(type='LoadFromVault', id='241025d7-3cca-4369-b7c0-b3264277c0e1', name='domostats_people', data_source_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None), DomoDataflow_Action(type='GenerateTableAction', id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', name='t_1', data_source_id=None, sql='SELECT\n`Display Name`,\n`User ID`\n, @row := @row+1 as id\n, @isReset := case when @iter = @rowCounter then 1 else 0 end isReset\n, @group := @isReset + @group   as rowGroup\n, @iter := case when @isReset = 1 then 1 else @iter + 1 end iter\n, @rowCounter := case when @isReset = 1 then FLOOR( RAND() * (7-3) + 3) else @rowCounter end numChildren\n\n\nfrom `domostats_people` p\n\n, (select @row:= 0,@rowCo

In [8]:
# | exporti
@patch_to(DomoDataflow)
async def execute(
    self: DomoDataflow, auth: dmda.DomoAuth = None, debug_api: bool = False
):
    return await dataflow_routes.execute_dataflow(
        auth=auth or self.auth, dataflow_id=self.id, debug_api=debug_api
    )

#### sample execute_dataflow


In [9]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataflow = DomoDataflow(id=70, auth=token_auth)

await domo_dataflow.execute()

ResponseGetData(status=200, response={'id': 345995, 'onboardFlowId': 70, 'previewRows': 0, 'dapDataFlowExecutionId': '29053f2b-840e-4394-bf96-4bd98c4635ac', 'beginTime': 1689697414175, 'lastUpdated': 1689697415039, 'state': 'CREATED', 'activationType': 'MANUAL', 'executionEngine': {'platform': 'K8S', 'engine': 'MYSQL_SQL_ENGINE'}, 'dataProcessor': 'MYSQL', 'dataFlowVersion': 262}, is_success=True)

In [10]:
# | hide
import nbdev

nbdev.nbdev_export()